> Copyright 2022 University of Luxembourg
> 
> Licensed under the Apache License, Version 2.0 (the "License");  
> you may not use this file except in compliance with the License.  
> You may obtain a copy of the License at  
>
>    https://www.apache.org/licenses/LICENSE-2.0
>
> Unless required by applicable law or agreed to in writing, software  
> distributed under the License is distributed on an "AS IS" BASIS,  
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
> See the License for the specific language governing permissions and  
> limitations under the License.  
>
***

Author: Andrzej Mizera (andrzej.mizera@uni.lu)

***

# AtMonSat Anomaly Detection Algorithm: training, validation, and testing of the deep-learning model

This notebook is used to train a deep-learning model for the anmonaly detection algorithm developed within the AtMonSat project. The trained model is saved in a .h5 file in the specified output directory.

Notebook content:
 - Determination of the threshold values for the Euclidean and Mahalanobis errors.
 - Model training.
 - Execution of the AtMonSat anomaly detection algorithm with the trained model on abnormal datasets.

---
### User settings

#### Window length

In [ ]:
window_length = 150

#### Deep-learning model to be used by the anomaly detection algorithm

See the aux_notebooks/models.ipynb notebook for details on the individual architectures.

In [ ]:
model_name = 'CNN'
#model_name = 'AutoEncoder'
#model_name = 'LSTM'
#model_name = 'LSTM_2'

#### Number of training epochs and the batch size

In [ ]:
epochs = 500
batch_size = 32

#### Name of the results directory

In [ ]:
output_folder = 'results_2022.11.14'

#### Normal datasets to be used for training and threshold values determination.

The experiment of 2022.07.20 contains only an abnormal dataset. Therefore, it is not on the list.

In [ ]:
use_datasets = ["2022.04.06", "2022.05.18", "2022.05.20", "2022.05.30", "2022.06.01",
                "2022.06.03", "2022.06.08", "2022.06.15", "2022.06.22"]

---

In [ ]:
# Print summary
print('Model:', model_name)
print('Window length:', window_length)
print('Number of epochs:', epochs)
print('Training batch size:', batch_size)

---
### Imports

In [ ]:
import os

import numpy as np

import math

import pandas as pd
from pandas import concat

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import metrics

from datetime import datetime
from copy import deepcopy
from tqdm.notebook import tqdm

---
### Internal settings - not to be modified by the User 

In [ ]:
time_format = 'timestamp'

DeltaLastChangeTimes_analysis = True

if DeltaLastChangeTimes_analysis:
    interpolation = False
    if interpolation:
        # Interpolation interval in seconds
        time_interpolation_interval = 5

normalise = True
# The 'Delta Last Change Times' transformed data should not be normalised.
normalise = normalise and (not DeltaLastChangeTimes_analysis)

PCA_higher_order_analysis = False
if PCA_higher_order_analysis:
    kPCA = False
    
# For anomaly_times variable to be defined.
anomaly_times = []

In [ ]:
# Print summary

print("DeltaLastChangeTimes_analysis:", DeltaLastChangeTimes_analysis)
if DeltaLastChangeTimes_analysis:
    print("\tInterpolation:", interpolation)
    if interpolation:
        print("\t\tTime interpolation interval:", str(time_interpolation_interval) + "s")
print("Normalise:",normalise)
print("PCA_higher_order_analysis:", PCA_higher_order_analysis)
if PCA_higher_order_analysis:
    print('\tKernel PCA:', kPCA)

In [ ]:
ANOMALY_COLOR = 'darkblue'
TP_INTERVAL_COLOR = ANOMALY_COLOR

In [ ]:
ISLC_implementation = ''
#ISLC_implementation = 'uint32'

In [ ]:
# MINMAXScaler or StandardScaler is now turned off!
apply_scaling = False

In [ ]:
data_path_common_prefix = 'datasets'
normal_dataset_file = os.path.join('processed_data','normal_data_temperature.csv')
abnormal_dataset_file = os.path.join('processed_data','anomalous_data_temperature.csv')

anomalies = {}

### Anomalies times
anomalies['2022.04.06'] = ['06.04.2022 16:36:47.00','06.04.2022 17:21:30.00','06.04.2022 17:50:59.00']
anomalies['2022.05.18'] = ['18.05.2022 16:14:24.00', '18.05.2022 16:56:19.00', '18.05.2022 17:44:32.00']
anomalies['2022.05.20'] = ['20.05.2022 19:14:37.00']
anomalies['2022.05.30'] = ['30.05.2022 18:26:24.00']
anomalies['2022.06.01'] = ['01.06.2022 12:49:21.00', '01.06.2022 14:26:10.00', '01.06.2022 16:06:33.00',
                           '01.06.2022 16:59:09.00']
anomalies['2022.06.03'] = ['03.06.2022 11:57:08.00']
anomalies['2022.06.08'] = ['08.06.2022 16:46:56.00', '08.06.2022 17:25:53.00']
anomalies['2022.06.15'] = ['15.06.2022 15:05:00.00', '15.06.2022 15:46:29.00']
anomalies['2022.06.22'] = ['22.06.2022 15:47:23.00']
anomalies['2022.07.20'] = ['20.07.2022 11:58:35.00', '20.07.2022 12:31:39.00', '20.07.2022 12:39:49.00',
                           '20.07.2022 14:38:26.00']

# Abnormal datasets trimming
# --------------------------
#  - assuring the initial part of length window_size is without anomalies;
#  - removing the end parts which contained noise due to experimental setup disassembly. 

new_anomalous_data_starts = {'2022.05.18':'18.05.2022 14:30:00.00', 
                             '2022.06.22':'22.06.2022 15:30:00.00'}

new_anomalous_data_ends = {'2022.06.15':'15.06.2022 16:20:00.00',
                           '2022.06.22':'22.06.2022 16:50:00.00'}

abnormal_datasets_dates = anomalies.keys()

In [ ]:
normal_features = ["temp_{}".format(i) for i in range(0,9)]
abnormal_features = ["temp_{}".format(i) for i in range(0,9)]

---
### Preparatory actions

#### Dataset class instantiation

The class is be used to load the normal datasets for model trainings.

In [ ]:
enable_example = False

%run dataset.ipynb

#### Create the results directory

In [ ]:
while True:
    
    CHECK_FOLDER = os.path.isdir(output_folder)
    
    if not CHECK_FOLDER:
        try:
            os.makedirs(output_folder)
            print("Directory '%s' created successfully" % output_folder)
            break
        except OSError as error:
            print(OSError)
            break
    else:
        print(output_folder, "directory already exists.")
        output_folder = input("Please provide a new directory name:\n")

#### Define auxiliary functions

In [ ]:
%run aux_notebooks/function_library.ipynb

In [ ]:
# load data
def parse(x):
    return pd.to_datetime(x, unit='s', origin='unix').round('S')
    
def parse_modified(x):
    return pd.to_datetime(x, origin='unix').round('S')

---
### Determination of the anomaly detection thresholds

In [ ]:
validation_data = []
mahalanobis_validation_data = []

# Cross-validation
for validation_dataset_date in use_datasets:
        
    print('Leave-one-out dataset:', validation_dataset_date)
    
    datasets_dates = use_datasets.copy()
    datasets_dates.remove(validation_dataset_date)
    print('Training datasets:', datasets_dates)
    
    datasets = []
    for dataset in datasets_dates:
        try:
            datasets.append(Dataset(dataset).extract('normal'))
        except Exception as e:
            print(e)
            
    VALIDATION_DATA = os.path.join(data_path_common_prefix,validation_dataset_date,normal_dataset_file)
    validation_df_full = pd.read_csv(VALIDATION_DATA,index_col=0,delimiter=',', parse_dates = ['timestamp'], date_parser=parse_modified)
    validation_df = validation_df_full
    df_valid_values = validation_df[normal_features].values
    
    %run aux_notebooks/error_thresholds_determination.ipynb
    
    validation_data.append(validation_error_data)
    mahalanobis_validation_data.append(mahalanobis_validation_error_data)

#### Plot histrograms and determine the threshold values

In [ ]:
def flatten(xss):
    return [x for xs in xss for x in xs]

In [ ]:
n, bins, patches = plt.hist(x=flatten(validation_data), bins='auto', color='#0504aa', alpha=0.7, rwidth=0.85)

In [ ]:
error_threshold = np.percentile(flatten(validation_data), 95)
print('Euclidean error threshold value:', error_threshold)

In [ ]:
fs = plt.rcParams.get('font.size')
plt.rcParams.update({'font.size': 15})
n, bins, patches = plt.hist(x=flatten(validation_data), bins='auto')
plt.vlines(error_threshold,0,400,colors='r',linestyles='dashed',label='Significance threshold')
plt.xlabel('Euclidean error')
plt.ylabel('Count')
plt.xticks(ticks=range(0,1500,200))
plt.legend()
plt.savefig(os.path.join(output_folder,'error_histogram.pdf'), format='pdf', bbox_inches='tight',
            pad_inches=0.1)
plt.show()
plt.rcParams.update({'font.size': fs})

In [ ]:
n, bins, patches = plt.hist(x=flatten(mahalanobis_validation_data), bins='auto', color='#0504aa', alpha=0.7, rwidth=0.85)

In [ ]:
#mahalanobis_error_threshold = np.percentile(flatten(mahalanobis_validation_data), 99)
mahalanobis_error_threshold = np.percentile(flatten(mahalanobis_validation_data), 95)
print('Mahalanobis error threshold value:', mahalanobis_error_threshold)

---
### Model training

In [ ]:
datasets = []
for dataset in datasets_dates:
    try:
        datasets.append(Dataset(dataset).extract('normal'))
    except Exception as e:
        print(e)

# Validation dataset for training
VALIDATION_DATASET = os.path.join(data_path_common_prefix,
                                  '2022.03.23','processed_data','validation_data_2022.03.23.csv')
validation_df_full = pd.read_csv(VALIDATION_DATASET, index_col=0, delimiter=',',
                                 parse_dates = ['timestamp'], date_parser=parse_modified)
validation_df = validation_df_full

%run aux_notebooks/train_model.ipynb

#### Perform post-processing of the Mahalanabis errors for the validation normal dataset.

In [ ]:
# The Mahalanobis errors are in m_dist computeted by aux_notebooks/train_model.ipynb.

anomaly_inertia_window_length = 60

# Consider a sequence of contiguous above-the-threshold values as a single anomaly detection signal 
# that indicates the occurance of the anomaly at the timepoint corresponding to the first
# above-the-threshold value in the sequence.
# Example: 0 0 0 1 1 1 0 0 1 1 0 -> 0 0 0 1 0 0 0 0 1 0 0
# To consider: Not first above-the-threshold value, but the max above-the-threshold value in a sequence.
    
pp_anomaly_detection_signal = np.zeros(len(m_dist))
pp_anomaly_signal_active = False

for ind_entry, entry in enumerate(m_dist >= mahalanobis_error_threshold):
        
    if entry and (not pp_anomaly_signal_active):

        pp_anomaly_detection_signal[ind_entry] = 1
        pp_anomaly_signal_active = True

    if pp_anomaly_signal_active and (not entry):
        pp_anomaly_signal_active = False
    
# Remove secondary anomaly detection signals, i.e., signals that appear within the 
# anomaly_inertia_window_length after an primary anomaly.
# Example: 0 0 0 1 0 0 1 0 0 1 0 0 0 -> 0 0 0 1 0 0 0 0 0 1 0 0 0
#                a - - - -    for anomaly_inertia_window_length = 4
i = 0
while i < len(pp_anomaly_detection_signal):
        
    if (pp_anomaly_detection_signal[i] == 1):
            
        j = 1
        while (j <= anomaly_inertia_window_length) and (i+j < len(pp_anomaly_detection_signal)):
            pp_anomaly_detection_signal[i+j] = 0
            j = j + 1
            
        i = i + j
        
    else:
        i = i + 1
    
print('Number of post-processed False Positives for the validation dataset:', sum(pp_anomaly_detection_signal))
print('Number of post-processed True Negatives for the validation dataset:',
      len(pp_anomaly_detection_signal) - sum(pp_anomaly_detection_signal) - window_length)

#### Plotting the output of the anomaly detection algorithm on the validation normal dataset.

In [ ]:
from matplotlib.dates import DateFormatter

fs = plt.rcParams.get('font.size')
plt.rcParams.update({'font.size': 20})
fig, ax = plt.subplots(figsize=(20, 2))
if DeltaLastChangeTimes_analysis:
    plt.plot(cet_valid, pp_anomaly_detection_signal, color='#16B9F5')
else:
    plt.plot(pp_anomaly_detection_signal, color='#16B9F5')
plt.yticks(ticks=[0,1],labels=['Norm','Abnorm'])

date_form = DateFormatter("%H:%M")
ax = plt.gca()
ax.xaxis.set_major_formatter(date_form)
plt.xlabel('Timestamp [HH:MM]',labelpad=20)

plt.savefig(os.path.join(output_folder,'validation_anomaly_detection.pdf'), format='pdf',
            bbox_inches='tight', pad_inches=0.1)
plt.show()
plt.rcParams.update({'font.size': fs})

#### Saving of the trained model and some variables to files.

In [ ]:
import pickle

inv_cov = np.linalg.inv(cov)

variables_dict = {'mahalanobis_mean':mean,
                  'mahalanobis_matrix':inv_cov,
                  'detection_threshold':mahalanobis_error_threshold,
                  'Euclidean_threshold':error_threshold
                 }

with open(os.path.join(output_folder,'saved_variables.pkl'), 'wb') as file:
    pickle.dump(variables_dict, file)

model.save(os.path.join(output_folder,'trained_model.h5'), save_format='h5')

---
#### Loading of a trained model and some variables.

In [ ]:
#import pickle

#MODEL_FOLDER = './results'

#with open(os.path.join(MODEL_FOLDER,'saved_variables.pkl'), 'rb') as file:
#    variables = pickle.load(file)
    
#mean = variables['mahalanobis_mean']
#mahalanobis_error_threshold = variables['detection_threshold']
#cov = np.linalg.inv(variables['mahalanobis_matrix'])
#error_threshold = variables['Euclidean_threshold']

#model = keras.models.load_model(os.path.join(MODEL_FOLDER,'./trained_model.h5'))

---
### The AtMonSat Anomaly Detection Algorithm

Besides running the algorithm itself, the code provides quantitative evaluation of the algorithm's performance.

In [ ]:
### Used for calculating raw and after post-processing TPs, FPs, TNs, and FNs.

def getConfusionTableValues(df,features,t_anomalies,anomaly_inertia_window_length,threshold,make_plot=False):

    ### Phase I of post-processing (only considered for pp_ prefixed values)
    # Consider a sequence of contiguous above-the-threshold values as a single anomaly detection signal 
    # that indicates the occurance of the anomaly at the timepoint corresponding to the first
    # above-the-threshold value in the sequence.
    # Example: 0 0 0 1 1 1 0 0 1 1 0 -> 0 0 0 1 0 0 0 0 1 0 0
    # To consider: Not first above-the-threshold value, but the max above-the-threshold value in a sequence.

    anomaly_detection_signal = np.zeros(len(m_dist))
    
    pp_anomaly_detection_signal = np.zeros(len(m_dist))
    pp_anomaly_signal_active = False

    for ind_entry, entry in enumerate(m_dist >= threshold):

        if entry:

            anomaly_detection_signal[ind_entry] = 1
        
        if entry and (not pp_anomaly_signal_active):

            pp_anomaly_detection_signal[ind_entry] = 1
            pp_anomaly_signal_active = True

        if pp_anomaly_signal_active and (not entry):
            pp_anomaly_signal_active = False
    
    ### Phase II of post-processing (only considered for pp_ prefixed values)
    # Remove secondary anomaly detection signals, i.e., signals that appear within the 
    # anomaly_inertia_window_length after an primary anomaly.
    # Example: 0 0 0 1 0 0 1 0 0 1 0 0 0 -> 0 0 0 1 0 0 0 0 0 1 0 0 0
    #                a - - - -    for anomaly_inertia_window_length = 4
    i = 0
    while i < len(pp_anomaly_detection_signal):
        
        if (pp_anomaly_detection_signal[i] == 1):
            
            j = 1
            while (j <= anomaly_inertia_window_length) and (i+j < len(pp_anomaly_detection_signal)):
                pp_anomaly_detection_signal[i+j] = 0
                j = j + 1
            
            i = i + j
        
        else:
            i = i + 1
        
    output = np.zeros(len(anomaly_detection_signal))
    pp_output = np.zeros(len(pp_anomaly_detection_signal))
    # 0 - TN, 1 - TP, -1 - FP
    
    FN = 0
    pp_FN = 0
    
    ### Count True Positives (TPs) and False Negatives (FNs):
    
    # Get a dictionary with t_anomalies as keys and pairs of anomaly index and the detection margin before
    # anomaly.
    interpInfo = getInterpolationInfoForAnomalies(df,features,t_anomalies)
    
    for t_anomaly in t_anomalies:
        
        # All anomaly detection signals in TP_interval should be considered as True Positives
        TP_interval = range(interpInfo[t_anomaly][0]-interpInfo[t_anomaly][1],
                            min(interpInfo[t_anomaly][0]+anomaly_inertia_window_length + 1,
                                len(df.index))
                           )
        
        anomaly_detected = False
        for i in TP_interval:
            if (anomaly_detection_signal[i] == 1):
                output[i] = 1
                anomaly_detected = True
        # Failed to detect the anomaly; increase the number of FNs
        if not anomaly_detected:
            FN = FN + 1
            
        pp_anomaly_detected = False
        for i in TP_interval:
            if (pp_anomaly_detection_signal[i] == 1):
                pp_output[i] = 1
                pp_anomaly_detected = True
        # Failed to detect the anomaly; increase the number of FNs
        if not pp_anomaly_detected:
            pp_FN = pp_FN + 1
    
    ### Count False Positives (FPs): 
    ### All signals in anomaly_detection_signal not tagged 1 in output are considered as FPs.
    for i in range(len(anomaly_detection_signal)):
        if (anomaly_detection_signal[i] == 1) and (output[i] == 0):
            output[i] = -1
            
    TP = sum(output == 1)
    FP = sum(output == -1)
    TN = len(anomaly_detection_signal) - (TP + FP + FN)
    
    for i in range(len(pp_anomaly_detection_signal)):
        if (pp_anomaly_detection_signal[i] == 1) and (pp_output[i] == 0):
            pp_output[i] = -1
            
    pp_TP = sum(pp_output == 1)
    pp_FP = sum(pp_output == -1)
    pp_TN = len(pp_anomaly_detection_signal) - (pp_TP + pp_FP + pp_FN)
    
    if make_plot:
        #figure(figsize=(20, 7))
        #plt.vlines(t_anomalies,0,1,color='b')
        #plt.plot(cet_abnorm, m_dist/max(m_dist), color='r')
        #plt.plot(df.index,output,color='g')
        #plt.show()
        
        fs = plt.rcParams.get('font.size')
        plt.rcParams.update({'font.size': 20})
        fig, ax = plt.subplots(figsize=(20, 2))
        plt.plot(cet_abnorm, pp_anomaly_detection_signal, color='#16B9F5')
        plt.vlines(t_anomalies,0,1,colors=ANOMALY_COLOR)
        for t_anomaly in t_anomalies:
            TP_interval = (interpInfo[t_anomaly][0]-interpInfo[t_anomaly][1],
                           min(interpInfo[t_anomaly][0]+anomaly_inertia_window_length + 1,len(df.index))
                          )
            start_time = cet_abnorm[TP_interval[0]]
            end_time = cet_abnorm[TP_interval[1]-1]
            ax.fill_between(cet_abnorm, 0, 1, where= (start_time <= cet_abnorm) & (cet_abnorm <= end_time),
                             facecolor=TP_INTERVAL_COLOR, alpha=0.15)
        plt.yticks(ticks=[0,1],labels=['Norm','Abnorm'])
        date_form = DateFormatter("%H:%M")
        ax = plt.gca()
        ax.xaxis.set_major_formatter(date_form)
        plt.xlabel('Timestamp [HH:MM]')
        plt.savefig(os.path.join(output_folder, 'anomaly_detection_' + anomaly_ds + '.pdf'), format='pdf',
                    bbox_inches='tight', pad_inches=0.1)
        plt.show()
        plt.rcParams.update({'font.size': fs})
    
    return {'TP':TP, 'FP':FP, 'TN':TN, 'FN':FN,
            'pp_TP':pp_TP, 'pp_FP':pp_FP, 'pp_TN':pp_TN, 'pp_FN':pp_FN}

In [ ]:
### For benchmark rule-based approach

temperature_change_window = anomaly_inertia_window_length

def checkTempIncrease(df):
    
    increased = []
    
    for col in df.columns:
        
        increase_found = False
        row = 0
        while (row < df.shape[0]-1):
            if (df[col].iloc[row] < df[col].iloc[row+1]):
                increase_found = True
            row = row + 1
                
        increased.append(increase_found)
        
    return np.array(increased).all()

def getAnomalyIntervals(predictions,anomaly_inertia_window_length):   
    predicted_anomaly_intervals = []

    anomaly_signal_active = False
    
    new_interval_found = False
    
    for i in range(len(predictions)):
        
        if (predictions[i] == 1):

            if not anomaly_signal_active:
                start_ind = i
                anomaly_signal_active = True

            if (i == len(predictions) - 1):
                end_ind = i
                new_interval_found = True

        else:

            if anomaly_signal_active:
                end_ind = i - 1
                new_interval_found = True
                anomaly_signal_active = False
                
        if new_interval_found:
            
            while (end_ind - start_ind + 1 > 2 * anomaly_inertia_window_length):
                
                predicted_anomaly_intervals.append((start_ind,start_ind + 2 * anomaly_inertia_window_length - 1))
                start_ind = start_ind + 2 * anomaly_inertia_window_length
                
            predicted_anomaly_intervals.append((start_ind,end_ind))
            
            new_interval_found = False
            
    return predicted_anomaly_intervals
    
# Test
# getAnomalyIntervals(np.array([0,0,1,1,1,0,0,0,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1]),3)

def getBenchmarkConfusionTableValues(predictions,anomaly_inertia_window_length,df,t_anomalies):
    
    ais = getAnomalyIntervals(predictions,anomaly_inertia_window_length)
    
    ind_anomalies = [list(df.index >= t_anomaly).index(True) for t_anomaly in t_anomalies]
    
    # It is assumed that the indices of the anomalies are monotonically increasing
    assert(all(ind_anomalies[i] <= ind_anomalies[i+1] for i in range(len(ind_anomalies) - 1)))
    
    TP = 0
    FP = 0
    FN = 0
    
    # i is the index iterating over the ind_anomalies list, which contains the positions (indices) of the
    # predictions array at which the anomalies were introduced.
    i = 0
    for interval in ais:
        
        # All anomalies in ind_anomalies were considered and/or there are no more anomalies to be detected.
        # Therefore, all the remaining intervals in ais are FPs.
        if (i == len(ind_anomalies)):
            
            FP = FP + 1
        
        while i < len(ind_anomalies):
            
            # The current interval starts after the currently considered anomaly and all the remaining
            # intervals start even later. Therefore, the current anomaly is not detected and the number
            # of FNs is increased.
            if (interval[0] > ind_anomalies[i]):
                
                FN = FN + 1
                i = i + 1
            
            # The current interval ends before the occurence of the currently considered anomaly and it
            # does not detect any of the previous anomalies. Therefore, it is a FP. i is not increased,
            # as one of the next intervals may be a detector of this anomaly - jump out of the while
            # loop and consider the next interval.
            if (interval[1] < ind_anomalies[i]):

                FP = FP + 1
                break
            
            # The currently considered anomaly lies within the current interval. This interval is a TP.
            if (interval[0] <= ind_anomalies[i]) and (ind_anomalies[i] <= interval[1]):
                
                TP = TP + 1
                i = i + 1
                
                # More than one anomaly could be correctly detected within the current interval. This
                # case is handled by the while loop.
                if (i == len(ind_anomalies)):
                    break
                else:
                    while (interval[0] <= ind_anomalies[i]) and (ind_anomalies[i] <= interval[1]):
                        TP = TP + 1
                        i = i + 1
                        if (i == len(ind_anomalies)):
                            break

                # Jump out of the while loop and condsider both the next interval and the next anomaly
                # (if exists).
                break

    # No intervals for the remaining anomalies, which are undetected and contribute to the number of FNs.
    while i < len(ind_anomalies):
        
        FN = FN + 1
        i = i + 1 
        
    TN = len(predictions)
    for interval in ais:
        TN = TN - (interval[1] - interval[0] + 1)

    return {'TP':TP, 'FP':FP, 'TN':TN, 'FN':FN}

In [ ]:
for anomaly_ds in abnormal_datasets_dates:
    
    print('Anomalous dataset:', anomaly_ds)
    
    ABNORMAL_DATA = os.path.join(data_path_common_prefix,anomaly_ds,abnormal_dataset_file)
    abnormal_df_full = pd.read_csv(ABNORMAL_DATA, index_col=0, delimiter=',',
                                   parse_dates = ['timestamp'], date_parser=parse_modified)
    abnormal_df = abnormal_df_full
        
    ### Apply trimming if necessary.
    new_start = new_anomalous_data_starts.get(anomaly_ds)
    new_end = new_anomalous_data_ends.get(anomaly_ds)

    if not new_start is None:
        t_new_start = datetime.strptime(new_start, '%d.%m.%Y %H:%M:%S.%f')
        t_new_start_ind = list(abnormal_df.index >= t_new_start).index(True)
        abnormal_df = abnormal_df.iloc[t_new_start_ind:,:]
        
    if not new_end is None:
        t_new_end = datetime.strptime(new_end, '%d.%m.%Y %H:%M:%S.%f')
        t_new_end_ind = list(abnormal_df.index > t_new_end).index(True)
        abnormal_df = abnormal_df.iloc[:t_new_end_ind,:]
    
    anomaly_times = anomalies[anomaly_ds]
    t_anomalies = [datetime.strptime(anomaly_time, '%d.%m.%Y %H:%M:%S.%f') for anomaly_time in anomaly_times]
    
    %run aux_notebooks/detect_anomalies.ipynb
    
    confusionTableDict = getConfusionTableValues(abnormal_df,abnormal_features,t_anomalies,
                                                 anomaly_inertia_window_length,mahalanobis_error_threshold,
                                                 make_plot=True
                                                )
    print('Confusion table for the Mahalanobis error threshold:', confusionTableDict)
    
    if (confusionTableDict['TP'] == 0):
        precision = 0
        recall = 0
        F1score = 0
    else:
        precision = confusionTableDict['TP']/(confusionTableDict['TP'] + confusionTableDict['FP'])
        recall = confusionTableDict['TP']/(confusionTableDict['TP'] + confusionTableDict['FN'])
        F1score = 2*(precision * recall) / (precision + recall)

    if (confusionTableDict['pp_TP'] == 0):
        pp_precision = 0
        pp_recall = 0
        pp_F1score = 0
    else:
        pp_precision = confusionTableDict['pp_TP']/(confusionTableDict['pp_TP'] + confusionTableDict['pp_FP'])
        pp_recall = confusionTableDict['pp_TP']/(confusionTableDict['pp_TP'] + confusionTableDict['pp_FN'])
        pp_F1score = 2*(pp_precision * pp_recall) / (pp_precision + pp_recall)
    
    print('Precision:', precision, '| Recall:', recall, '| F1 score:', F1score)
    print('pp_Precision:', pp_precision, '| pp_Recall:', pp_recall, '| pp_F1 score:', pp_F1score)
    
    # Thresholds are sorted in the increasing order by np.unique() and then reverted. In result,
    # the thresholds are sorted in the decreasing order.
    #
    # As in sklearn.metrics.roc_curve(), thresholds[0] represents no instances being predicted
    # and is arbitrarily set to max(m_dist) + 1. This is to make sure that the curve starts at (0, 0).
    thresholds = np.append(np.unique(m_dist), max(m_dist)+1)[::-1]
    assert(all(thresholds[i] >= thresholds[i+1] for i in range(len(thresholds) - 1)))

    FPRs = []
    TPRs = []

    pp_FPRs = []
    pp_TPRs = []

    precisions = []
    recalls = []

    pp_precisions = []
    pp_recalls = []

    #for thr in tqdm(thresholds):
    for i in tqdm(range(len(thresholds))):

        thr = thresholds[i]

        #confusionTableDict = getConfusionTableValues(abnormal_df,abnormal_features,t_anomalies,
        #                                             anomaly_inertia_window_length,thr)

        confusionTableDict = getConfusionTableValues(abnormal_df,abnormal_features,t_anomalies,
                                                     anomaly_inertia_window_length,thr)

        pp_TP = confusionTableDict['pp_TP']
        pp_FN = confusionTableDict['pp_FN']
        pp_FP = confusionTableDict['pp_FP']
        pp_TN = confusionTableDict['pp_TN']

        pp_fpr = pp_FP/(pp_FP+pp_TN)
        pp_tpr = pp_TP/(pp_TP+pp_FN)

        pp_FPRs.append(pp_fpr)
        pp_TPRs.append(pp_tpr)

        # The first threshold value, i.e., max(m_dist)+1, is used only for the ROC curve.
        if (i != 0):
            pp_precision = pp_TP/(pp_TP + pp_FP)
            pp_recall = pp_TP/(pp_TP + pp_FN)

            pp_precisions.append(pp_precision)
            pp_recalls.append(pp_recall)

        TP = confusionTableDict['TP']
        FN = confusionTableDict['FN']
        FP = confusionTableDict['FP']
        TN = confusionTableDict['TN']

        fpr = FP/(FP+TN)
        tpr = TP/(TP+FN)

        FPRs.append(fpr)
        TPRs.append(tpr)

        # The first threshold value, i.e., max(m_dist)+1, is used only for the ROC curve.
        if (i != 0):
            precision = TP/(TP + FP)
            recall = TP/(TP + FN)

            precisions.append(precision)
            recalls.append(recall)


    # Reverse the precisions and recalls so recalls is decreasing.
    #
    # As in the case of sklearn.metrics.precision_recall_curve(), the last precision and recall values
    # are 1. and 0. respectively and do not have a corresponding threshold. This ensures that the graph
    # starts on the y axis.

    sl = slice(None, None, -1)
    precisions = precisions[sl]
    precisions.append(1)
    recalls = recalls[sl]
    recalls.append(0)

    assert(all(recalls[i] >= recalls[i+1] for i in range(len(recalls) - 1)))

    pp_precisions = pp_precisions[sl]
    pp_precisions.append(1)
    pp_recalls = pp_recalls[sl]
    pp_recalls.append(0)

    roc_auc = auc(FPRs, TPRs)

    pr_auc = auc(recalls, precisions)

    plt.figure()
    lw = 2
    plt.plot(
        FPRs,
        TPRs,
        color="darkorange",
        lw=lw,
        label="ROC curve (area = %0.2f)" % roc_auc,
    )
    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    #plt.title("Receiver operating characteristic curve")
    plt.legend(loc="lower right",fontsize=16)
    
    plot_file_name = os.path.join(output_folder,'ROC_curve_' + anomaly_ds + '.pdf')
    plt.savefig(plot_file_name, format='pdf', bbox_inches='tight', pad_inches=0.1)
    
    plt.show()

    # https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-imbalanced-classification/
    # Both the precision and the recall are focused on the positive class (the minority class) and are 
    # unconcerned with the true negatives (majority class).
    #
    #    "… precision and recall make it possible to assess the performance of a classifier
    #     on the minority class."
    #
    #       — Page 27, Imbalanced Learning: Foundations, Algorithms, and Applications, 2013.
    #
    #    "Precision-recall curves (PR curves) are recommended for highly skewed domains where
    #     ROC curves may provide an excessively optimistic view of the performance."
    #
    #       — A Survey of Predictive Modelling under Imbalanced Distributions, 2015.

    # calculate the no skill line as the proportion of the positive class
    no_skill = len(t_anomalies)/len(m_dist)
    # plot the no skill precision-recall curve
    fs = plt.rcParams.get('font.size')
    plt.rcParams.update({'font.size': 20})
    plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
    plt.plot(recalls, precisions, marker='.', label="Precision-Recall curve (area = %0.2f)" % pr_auc)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend(fontsize=12)
    
    plot_file_name = os.path.join(output_folder,'Precision-Recall_curve_' + anomaly_ds + '.pdf')
    plt.savefig(plot_file_name, format='pdf', bbox_inches='tight', pad_inches=0.1)
  
    plt.show()
    plt.rcParams.update({'font.size': fs})
    
    ### Benchmark rule-based method for comparison
    ### ==========================================
    
    print("========= Benchmark rule-based method =========")
    
    anomaly_detection = []
    for i in range(abnormal_df.shape[0]-temperature_change_window):
        if checkTempIncrease(abnormal_df[abnormal_features].iloc[i:i+temperature_change_window]):
            anomaly_detection.append(1)
        else:
            anomaly_detection.append(0)

    for i in range(temperature_change_window):
        if checkTempIncrease(abnormal_df[abnormal_features].iloc[abnormal_df.shape[0]-temperature_change_window+i:]):
            anomaly_detection.append(1)
        else:
            anomaly_detection.append(0)

    fs = plt.rcParams.get('font.size')
    plt.rcParams.update({'font.size': 20})
    figure(figsize=(20, 2))
    plt.plot(abnormal_df.index,anomaly_detection, color='#16B9F5')
    plt.vlines(t_anomalies,0,1,colors=ANOMALY_COLOR)
    plt.yticks(ticks=[0,1],labels=['Norm','Abnorm'])
    date_form = DateFormatter("%H:%M")
    ax = plt.gca()
    ax.xaxis.set_major_formatter(date_form)
    plt.xlabel('Timestamp [HH:MM]',labelpad=20)
    
    plot_file_name = os.path.join(output_folder,'benchmark_anomaly_detection_' + anomaly_ds + '.pdf')
    plt.savefig(plot_file_name, format='pdf', bbox_inches='tight', pad_inches=0.1)

    plt.show()
    plt.rcParams.update({'font.size': fs})
    
    print("Confusion matrix of the benchmark rule-based method:")
    bm_confusionTableDict = getBenchmarkConfusionTableValues(anomaly_detection,temperature_change_window,abnormal_df,t_anomalies)
    print(bm_confusionTableDict)
    
    if (bm_confusionTableDict['TP'] == 0):
        bm_precision = 0
        bm_recall = 0
        bm_F1score = 0
    else:
        bm_precision = bm_confusionTableDict['TP']/(bm_confusionTableDict['TP'] + bm_confusionTableDict['FP'])
        bm_recall = bm_confusionTableDict['TP']/(bm_confusionTableDict['TP'] + bm_confusionTableDict['FN'])
        bm_F1score = 2*(bm_precision * bm_recall) / (bm_precision + bm_recall)
    
    print('Benchmark precision:', bm_precision, '| Benchmark recall:', bm_recall,
          '| Benchmark F1 score:', bm_F1score)
    
    print("====================================================================================================\n")